<h2> Only Input Criteria is the "folder_name" </h2>

In [ ]:
################################################
#### BELOW IS ONLY ITEM TO EDIT IN SCRIPT ######
########## IDENTIFIES WHERE FILES ARE ##########
################################################

folder_name = "FTD"


## WITHIN THE TOOL THERE SHOULD ALSO BE A WAY TO DROP FILES INTO "client_files" and "competitive_files" ##

In [16]:
# STEP #1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# STEP #2: Insert the directory
import sys
sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks/SEO Work/GAP Analysis Script - SEMRush')

import pandas as pd
import glob
from urllib.parse import urlparse
from datetime import datetime
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
#### BELOW IDENTIFIES PATH FOR FOLDERS ####
colab_path = "/content/drive/My Drive/Colab Notebooks/SEO Work/GAP Analysis Script - SEMRush"
client_files = glob.glob(colab_path + "/Client SEMRush (GAP)/{}/*.csv".format(folder_name))
competitive_files = glob.glob(colab_path + "/Competitive SEMRush (GAP)/{}/*.csv".format(folder_name))


### BELOW CREATED DATAFRAME FOR THE CLIENT(S) SEMRUSH DATASETS ###
### CAN BE ONE OR MULTIPLE CSV'S ###

li_client = []

for filename in client_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li_client.append(df)

df_clients = pd.concat(li_client, axis=0, ignore_index=True)


### BELOW CREATED DATAFRAME FOR THE COMPETITOR(S) SEMRUSH DATASETS ###
### CAN BE ONE OR MULTIPLE CSV'S ###

li_competitors = []

for filename in competitive_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li_competitors.append(df)

df_competitors = pd.concat(li_competitors, axis=0, ignore_index=True)

In [18]:
### BELOW MAKES SOME MODIFICATIONS TO THE COMPETITOR(S) DATASETS ###

def position_range(position):
    if position < 11:
        return "1 to 10"
    elif position >= 11 and position <= 20:
        return "11 to 20"
    elif position >= 21 and position <= 30:
        return "11 to 20"
    else:
        return "31+"
    
def top20(position):
    if position < 21:
        return "TRUE"
    else:
        return "FALSE"
    
def traffic_exists(traffic):
    if traffic > 0:
        return "TRUE"
    else:
        return "FALSE"

domain_name = df_competitors['URL'].str.extract('(?<=http://)(.*?)(?=/)|(?<=https://)(.*?)(?=/)')
domain_name = domain_name[0].fillna(domain_name[1]).fillna(df_competitors['URL'])

df_competitors.insert(loc = 2, column = "Position_Range", value = df_competitors["Position"].apply(position_range))
df_competitors.insert(loc = 3, column = "Top20", value = df_competitors["Position"].apply(top20))
df_competitors.insert(loc = 7, column = "Domain", value = domain_name)
df_competitors.insert(loc = 11, column = "Traffic_Exists", value = df_competitors["Traffic"].apply(traffic_exists))
df_competitors.insert(loc = 11, column = "Traffic_Rank", value = df_competitors["Traffic"].rank(ascending = False).astype(int))

In [19]:
### BELOW MERGES THE COMPETITOR DATASET w/ CLIENTS AND CLEANS COLUMN NAMES ###

df_gap = df_competitors.merge(df_clients, how="left", left_on=["Keyword"], right_on=["Keyword"])

select_cols = ["Keyword","Search Volume_x", "Keyword Difficulty_x", "CPC_x","Position_x", "Position_y",  "Top20", "Position_Range","Traffic_x", "Traffic Cost_x", "URL_x","Domain" ,"URL_y" ]
df_gap = df_gap[select_cols]

df_gap = df_gap.rename(
    mapper={
        "Keyword": "Keyword",
        "Search Volume_x": "Search_Volume",
        "Keyword Difficulty_x": "KD",
        "CPC_x": "CPC",

        "Position_x": "Position_Comp",
        "Position_y": "Position_Client",
        "Position_Range": "Position_Range_Comp",
        "Traffic_x": "Traffic_Comp",
        "Top20": "Top20_Comp",
        "Traffic Cost_x": "Traffic_Cost_Comp",


        "URL_x": "URL_Comp",
        "Domain": "Domain",
        "URL_y": "URL_Client",


        "clicks_sum": "clicks_sum_gsc",
        "impressions_sum": "impressions_sum_gsc",
        "ctr_mean": "ctr_mean_gsc",
        "position_size": "count_instances_gsc",
        "position_max": "position_max_gsc",
        "position_min": "position_min_gsc",
        "position_mean": "position_mean_gsc",
        "text": "text_crawl",
        "occurrences": "occurrences_crawl"
    }, axis="columns")

df_gap = df_gap.fillna(value = "Not Ranking")

In [20]:
##### BELOW SENDS FINAL DATASET TO CSV + PRINT'S RESULTS FOR REFERENCE #####

date = datetime.today().strftime('%Y-%m-%d')
df_gap.to_csv(os.path.join(colab_path,"GAP_Analysis_{}-{}.csv".format(folder_name,date)),  index=False)

df_gap

,Keyword,Search_Volume,KD,CPC,Position_Comp,Position_Client,Top20_Comp,Position_Range_Comp,Traffic_Comp,Traffic_Cost_Comp,URL_Comp,Domain,URL_Client
0,flowers,1830000,81.29,2.47,1,3,TRUE,1 to 10,860100,2124447.0,https://www.1800flowers.com/,www.1800flowers.com,https://www.ftd.com/
1,flower delivery,673000,82.67,3.43,1,5,TRUE,1 to 10,316310,1084943.0,https://www.1800flowers.com/flowers-same-day-d...,www.1800flowers.com,https://www.ftd.com/flowers/best-sellers
2,mothers day flowers,110000,79.45,1.69,1,3,TRUE,1 to 10,51700,87373.0,https://www.1800flowers.com/mothers-day-flowers,www.1800flowers.com,https://www.ftd.com/mothers-day/flowers
3,flower bouquet,90500,84.38,1.51,1,2,TRUE,1 to 10,42535,64227.0,https://www.1800flowers.com/mixedbouquets,www.1800flowers.com,https://www.ftd.com/flowers/all
4,flower bouquet,90500,84.38,1.51,1,3,TRUE,1 to 10,42535,64227.0,https://www.1800flowers.com/mixedbouquets,www.1800flowers.com,https://www.ftd.com/flowers/best-sellers
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92476,4 year anniversary gift for her,590,83.94,0.84,90,Not Ranking,FALSE,31+,0,0.0,https://www.teleflora.com/gift-giving-ideas/an...,www.teleflora.com,Not Ranking
92477,cheapest flower delivery in india,30,58.83,0.00,33,59,FALSE,31+,0,0.0,https://www.teleflora.com/,www.teleflora.com,https://www.ftd.com/
92478,sunny smiles,1900,62.15,2.14,20,Not Ranking,TRUE,11 to 20,5,12.0,https://www.teleflora.com/bouquet/telefloras-s...,www.teleflora.com,Not Ranking
92479,non flower sympathy gifts,30,76.09,1.19,89,24,FALSE,31+,0,0.0,https://www.teleflora.com/funeral-sympathy-col...,www.teleflora.com,https://www.ftd.com/occasion/sympathy/gift-bas...
